In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm_notebook
import xgboost as xgb

from tools.data_cleansing import clean
from tools.data_exploration import print_email_by_id, emails_sent_distribution, emails_received_distribution,\
body_length_distribution, number_of_recipients_distribution
from tools.data_handling import enrich_emails, load_email_senders, unique_recipients, address_book, load_emails
from tools.features import split_tokenizer, TwidfVectorizer, TeamVectorizer, FastCountVectorizer
from tools.sender_pipeline import SenderModel
from tools.submission import save_submission
from tools.staff_graph import compute_summary_graph, construct_graph, compute_teams, assign_team

SyntaxError: invalid syntax (staff_graph.py, line 143)

# Data loading and first look

In [4]:
df_emails = enrich_emails()

NameError: name 'enrich_emails' is not defined

In [ ]:
print_email_by_id(df_emails, 265941)

In [ ]:
df_email_senders = load_email_senders()

# Data exploration

In [ ]:
emails_sent_distribution(df_email_senders, max_value=300)

In [ ]:
emails_received_distribution(df_emails, max_value=200)

In [ ]:
body_length_distribution(df_emails, max_value=10000)

In [ ]:
number_of_recipients_distribution(df_emails, bins=20, max_value=100)

In [ ]:
unique_rec = unique_recipients(df_emails)
print("Number of unique recipients: {}".format(len(unique_rec)))

In [ ]:
df_emails[df_emails["sender"] == email]

# Testing TeamVectorizer

In [ ]:
team_vectorizer = TeamVectorizer()

In [ ]:
team_vectorizer.fit(df_emails)

In [ ]:
team_vectorizer.n_features

In [ ]:
a = team_vectorizer.transform(["steven.january@enron.com", "terry.kowalke@enron.com", "lynn.blair@enron.com"])

In [ ]:
b = team_vectorizer.transform(["rick.dietz@enron.com", "raetta.zadow@enron.com", "larry.berger@enron.com"])

# Data cleansing

## Cleansing function

In [ ]:
def clean_df(df_emails, df_senders):
    df_emails["clean_body"] = ""
    for index, row in tqdm_notebook(df_senders.iterrows(), desc="Senders cleaning", total=df_senders.shape[0]):
        mids = list(map(int, row["mids"].split()))
        # data loading and separation
        df_interest = df_emails.ix[mids]
        # data cleansing
        unique_rec_train = unique_recipients(df_interest)
        add_book = address_book(unique_rec_train)
        df_emails.loc[mids, "clean_body"] = df_interest["body"].apply(lambda x: clean(x, add_book))
    df_emails["clean_body"] = df_emails["clean_body"].fillna("")

In [ ]:
clean_df(df_emails, df_email_senders)

In [ ]:
df_emails[df_emails["body"].str.contains("Forwarded by")].head(20)

# Training

In [ ]:
df_small_senders = df_email_senders.sample(frac=0.3, random_state=0)

## Current Model

In [ ]:
def model(df_interest, n_mails):
    # Input TF-IDF vectorizer
    input_bow = FastCountVectorizer()
    # Random Forest Regressor
    rf = RandomForestRegressor(
        n_estimators=190,
        max_depth=75,
        n_jobs=-1,
        min_samples_leaf=max(1,int(0.0002*n_mails)),
        max_features="sqrt",
        random_state=0)
    # Output vectorizer
    unique_rec = unique_recipients(df_interest)
    output_bow = CountVectorizer(
        tokenizer=split_tokenizer,
        vocabulary=unique_rec)
    # Sender Model
    sender_model = SenderModel(
        df_emails=df_interest,
        classifier = rf,
        input_vectorizer = input_bow,
        output_vectorizer = output_bow
    )
    return sender_model

## On one particular sender

In [ ]:
row = df_email_senders.iloc[0]
mids = list(map(int, row["mids"].split()))
df_interest = df_emails.ix[mids]
n_mails = len(mids)
sender_model = model(df_interest, n_mails)
prec = sender_model.evaluate(random_state=0)
print(prec)

## Evaluation

In [ ]:
df_precision = pd.DataFrame(columns=["sender", "n_mails", "precision"])

In [ ]:
for index, row in tqdm_notebook(df_email_senders.iterrows(), desc="Senders evaluation", total=df_email_senders.shape[0]):
    # Row unpacking
    sender = row["sender"]
    mids = list(map(int, row["mids"].split()))
    df_interest = df_emails.ix[mids]
    n_mails = len(mids)
    # Model building
    sender_model = model(df_interest, n_mails)
    # Precision computation
    prec = sender_model.evaluate(random_state=0)
    df_precision.loc[index] = [sender, n_mails, prec]

In [ ]:
prec

In [ ]:
(df_precision["precision"]*df_precision["n_mails"]).sum()/(df_precision["n_mails"].sum())

## Testing function

In [ ]:
def add_team_info(sender, predicted_recipients, teams, n_clusters):
        ''' Reorder the ten email predicted with defined from heuristic
        Arguments:
            - sender (str): email from the sender
            - predicted_recipients (np.array): Top predicted emails (strings)
                                   shape = n_samples, top
            - teams: dict of teams for all the email adresses
            - n_clusters: number of clusters in the Louvain assignment
        Output:
            - new_prediction (np.array): team updated prediction.
        '''
        sender_team = teams[sender]
        recipients_teams = np.zeros_like(predicted_recipients)
        new_prediction = np.zeros_like(predicted_recipients)
        for raw in range(predicted_recipients.shape[0]):
            current_rank = 0
            in_new_pred = zeros(predicted_recipients.shape[1])
            for email_index in range(predicted_recipients.shape[1]):
                if predicted_recipients[raw, email_index] in teams.keys():
                    recipients_teams[raw, email_index] = teams[
                        predicted_recipients[raw, email_index]]
                else:
                    recipients_teams[raw, email_index] = 0
                # people in the same team than sender are First
                if recipients_teams[raw, email_index] == sender_team:
                    new_prediction[raw, current_rank] = predicted_recipients[
                        raw, email_index]
                    current_rank += 1
                    in_new_pred[email_index] = 1
            counts = np.bincount(recipients_teams)
            max_team = np.argmax(counts)
            # put all people from max_team second
            if max_team != 0:
                for email_index in range(predicted_recipients.shape[1]):
                    if recipients_teams[raw, email_index] == max_team:
                        new_prediction[raw, current_rank] =\
                                predicted_recipients[raw, email_index]
                        current_rank += 1
            for email_index in range(predicted_recipients.shape[1]):
                if in_new_pred[email_index] == 0:
                    new_prediction[raw, current_rank] = predicted_recipients[
                        raw, email_index]
                    current_rank += 1
                    in_new_pred[email_index] = 1

            # sanity_check
            if np.sum(in_new_pred) != predicted_recipients.shape[1]:
                print("warning: not all emails where mapped")
        return new_prediction

In [ ]:
team_vectorizer = TeamVectorizer()
team_vectorizer.fit(df_emails)

In [ ]:
add_team_info(sender, predicted_recipients, teams, n_clusters)

# Submission

## Actual model training

In [ ]:
sender_models = dict()
for index, row in tqdm_notebook(df_email_senders.iterrows(), desc="Senders Training (submision)", total=df_email_senders.shape[0]):
    # Row unpacking
    sender = row["sender"]
    mids = list(map(int, row["mids"].split()))
    df_interest = df_emails.ix[mids]
    n_mails = len(mids)
    # Model building
    sender_model = model(df_interest, n_mails)
    # Training
    sender_model.train()
    # Saving
    sender_models[sender] = sender_model

## Data loading

In [ ]:
df_submission_senders = load_email_senders(set_type="test")
df_submission = load_emails(set_type="test")
df_submission["recipients"] = ""

## Data cleansing

In [ ]:
clean_df(df_submission, df_submission_senders)

## Prediction

In [ ]:
for index, row in tqdm_notebook(df_submission_senders.iterrows(), desc="Senders predictions", total=df_submission_senders.shape[0]):
    sender = row["sender"]
    mids = map(int, row["mids"].split())
    # Model loading
    sender_model = sender_models[sender]
    # Prediction
    sender_model.predict(mids, df_submission)

# Fine tuning

In [ ]:
from tools.fine_tuning import expected_precision

In [ ]:
depths = [2, 5, 10, 15, 20, 30, 50, 100, 300]
n = len(depths)
p = np.zeros(n)

In [ ]:
for i, depth in enumerate(depths):
    p[i] = expected_precision(min_sample_prop=0.0002,
                             n_estimators=15,
                             max_depth=100)

In [ ]:
n_trials = 10
pr = np.zeros(n_trials)
for i in range(n_trials):
    pr[i] += expected_precision(min_sample_prop=0.04,
                             n_estimators=15,
                             max_depth=30)
print(pr)

In [ ]:
pr.mean()

In [ ]:
plt.plot(depths, p)
plt.xlabel("Number of trees")
plt.ylabel("Precision")

## Formatting

In [ ]:
save_submission(df_submission,
               algo="RF single sender regressor tw idf",
               member="Zac")

In [ ]:
df_submission